In [1]:
import numpy as np
import pandas as pd


In [2]:
def eval_one(f_name):
    df_raw = pd.read_csv(f_name, skiprows=6)
    df = df_raw[[
        '[run number]',
        '[step]',
        'swap-to-hate',                             # Society becomes hateful when > 30% hateful nodes
        # 'p_defer_hateful_post',                     # Deferring probability for hateful posts
        'hate_score_dist_alpha',                    # Alpha-parameter of the Gamma distribution

        'max-out-degree false',                     # Max degree of normals
        'max-out-degree true',                      # Max degree of haters
        'connection-density false',                 # Connection density of normals
        'connection-density true',                  # Connection density of haters
        'reciprocal-connections-percents false',    # Reciprocity of normals
        'reciprocal-connections-percents true',     # Reciprocity of haters
        'mean-follower-followee false',             # Follower/Followee of normals
        'mean-follower-followee true',              # Follower/Followee of haters
        'mean-following true false',                # Mean followers of normals
        'mean-following true true',                 # Mean followers of haters
        'mean-following false false',               # Mean followees of normals
        'mean-following false true',                # Mean followees of haters
        # 'mean-follow-prob true false',            # Mean following probabilities of hater to normal
        'mean-follow-prob true true',               # Mean following probabilities of hater to hater
        'mean-follow-prob false false',             # Mean following probabilities of normal to normal
        # 'mean-follow-prob false true',            # Mean following probabilities of normal to hater
        'connections-percents true false',          # Fraction of edges from haters to normals
        'connections-percents false true',          # Fraction of edges from normals to haters
        'percents-persons-with-degree false true 0',# Fraction of normals with out-degree 0
        'percents-persons-with-degree false true 1',# Fraction of normals with out-degree 0
        'percents-persons-with-degree true true 0', # Fraction of haters with out-degree 0
        'percents-persons-with-degree true true 1', # Fraction of haters with out-degree 0
        'percents-hateful-persons',                 # % of hateful users
        'percents-hateful-posts',                   # % of hateful posts
        'stddev-hate-score-dist',                   # StdDev of hate score distribution
        'mean-hate-score',                          # Mean of hate score distribution
        'reposts-by-haters',                        # Reposts by haters
        'reposts-by-normals',                       # Reposts by normals
        'mean-path-length false',                   # Mean post path length of normal posts
        'mean-path-length true',                    # Mean post path length of hateful posts
    ]]

    df_swap = df.copy()
    df = df[df['swap-to-hate'] != True]

    df['percents-hateful-persons'] /= 100

    # Compute ratio of connection densities
    df['density'] = df['connection-density true'] / df['connection-density false']

    # Compute ratio of NH / HN edges
    df['connections'] = df['connections-percents false true'] / df['connections-percents true false']

    # Compute ratio of max out-degrees
    df['ratio-max-out-degrees'] = df[df['max-out-degree true'] > 0]['max-out-degree false'] / df[df['max-out-degree true'] > 0]['max-out-degree true']

    df_all = df.groupby(['hate_score_dist_alpha']).mean()

    df_plot = df_all[[
        'percents-hateful-persons',
        'percents-hateful-posts',
        'mean-hate-score',
        'stddev-hate-score-dist',

        'density',
        'ratio-max-out-degrees',
        'reciprocal-connections-percents false',
        'reciprocal-connections-percents true',

        'mean-following true false',
        'mean-following true true',
        'mean-following false false',
        'mean-following false true',
        'mean-follower-followee false',
        'mean-follower-followee true',

        'mean-path-length false',
        'mean-path-length true',
        'reposts-by-normals',
        'reposts-by-haters',
    ]]
    df_plot.columns = [
        'Fraction of hateful persons',
        'Fraction of hateful posts',
        'Mean of hate score distribution',
        'StdDev of hate score distribution',

        'Ratio network density H - N',
        'Ratio max. followers of N - H',
        'Reciprocity of normals',
        'Reciprocity of haters',

        'Mean followers of normals',
        'Mean followers of haters',
        'Mean followees of normals',
        'Mean followees of hater',
        'Mean follower-followee of normals',
        'Mean follower-followee of haters',

        'Mean path len normals',
        'Mean path len haters',
        'Reposts of normals',
        'Reposts of haters',
    ]
    df_plot['Swap to hate'] = df_swap.groupby(['hate_score_dist_alpha'])['swap-to-hate'].mean()


    return df_plot

In [3]:

f_name_prefix = 'evaluation_csv/network_growth validate_counter_education_'
f_name_suffix = '-table.csv'

n_sizes = [
    '0_1000',
    '500_1500',
    '1000_2000',
    '2000_3000',
    '5000_6000',
    # '10000_11000',
]

gammas = [
    '2',
    '4',
    '6',
    '8',
    # '8_a',
]


get_col = 'Fraction of hateful persons'
# get_col = 'Fraction of hateful posts'
# get_col = 'Mean of hate score distribution'
# get_col = 'StdDev of hate score distribution'

# get_col = 'Ratio network density H - N'
# get_col = 'Ratio max. followers of N - H'
# get_col = 'Reciprocity of normals'
# get_col = 'Reciprocity of haters'

# get_col = 'Mean followers of normals'
# get_col = 'Mean followers of haters'
# get_col = 'Mean followees of normals'
# get_col = 'Mean followees of hater'
# get_col = 'Mean follower-followee of normals'
# get_col = 'Mean follower-followee of haters'

# get_col = 'Mean path len normals'
# get_col = 'Mean path len haters'
# get_col = 'Swap to hate'

df_collect = pd.DataFrame(columns=['hate_score_dist_alpha', *list(n_sizes)])
df_collect.hate_score_dist_alpha = gammas

for s in n_sizes:
    df_new = pd.DataFrame()
    for g in gammas:
        f_name = f_name_prefix + s + '-' + str(g) + f_name_suffix
        df = eval_one(f_name)
        df_new = df_new.append(df)

    df_collect.loc[:, s] = list(df_new[get_col])


df_collect = df_collect.sort_values('hate_score_dist_alpha', ascending=False)
df_collect.to_csv('summary_counter_education_' + get_col + '.csv', index=False)
df_collect

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,hate_score_dist_alpha,0_1000,500_1500,1000_2000,2000_3000,5000_6000
3,8,0.028033,0.038858,0.042851,0.043835,0.056904
2,6,0.023162,0.020378,0.026658,0.031062,0.032098
1,4,0.017343,0.017880,0.019255,0.020184,0.022597
0,2,0.014270,0.014721,0.015112,0.016177,0.016941


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=76a22323-d43c-4823-9752-036f6744cf38' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>